In [1]:
import os
from tabulate import tabulate

# Define a function to count PDF files in a folder
def count_pdf_files(folder):
    pdf_count = 0
    for file in os.listdir(folder):
        if file.endswith('.pdf'):
            pdf_count += 1
    return pdf_count

# Define the root directory where the folders are located
root_directory = 'data/raw data'  # Atualizado com o novo caminho

# List all directories in the root directory
directories = [d for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d))]

# Create a list to store folder names and their respective PDF counts
folder_pdf_counts = []

# Iterate over each directory
for directory in directories:
    pdf_count = count_pdf_files(os.path.join(root_directory, directory))
    folder_pdf_counts.append([directory, pdf_count])

# Sort the folder PDF counts list by PDF count
folder_pdf_counts.sort(key=lambda x: x[1], reverse=True)

# Print the table
print(tabulate(folder_pdf_counts, headers=['Folder Name', 'PDF Count']))


Folder Name                            PDF Count
-----------------------------------  -----------
SICREDI CAMPOS GERAIS E GRANDE CURI           89
COOP SICREDI ARAXINGÚ                         31
SICREDI CEARÁ                                 28
COOP SICREDI NORTE                            26
COOP SICREDI NORTE SC                         26
COOP SICREDI CELEIRO OESTE                    25
COOP SICREDI UNIÃO METROPOLITANA              21
COOP SICREDI EVOLUÇÃO                         19
COOP SICREDI VALE SÃO FRANCISCO               18
COOP SICREDI SUDOESTE MT_PA                   17
CC SICREDI COOMAMP                            16
COOP SICREDI FORÇA DOS VENTOS                 16
COOP SICREDI ROTA DAS TERRAS RS_MG            15
COOP SICREDI ARACAJU                          13
COOP SICREDI CAMPO GRANDE                     13
COOP SICREDI INTEGRAÇÃO RS_MG                 13
SICREDI CREDUNI                               13
COOP SICREDI CENT PERNAMBUCANA                10
COOP SICREDI CULTURA

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     --------------------------------- ---- 204.8/232.6 kB 6.1 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
#pip install pdf2image
#pip install poppler-utils
#pip install pytesseract
#pip install tesseract
# !pip install tesseract-ocr
# !pip install pytesseract
#!pip install pytesseract
#pip install PyPDF2





[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
import pytesseract
from pdf2image import convert_from_path
from PyPDF2 import PdfReader

# Defina o diretório TESSDATA_PREFIX para apontar para o diretório pai do diretório tessdata
os.environ['TESSDATA_PREFIX'] = r'C:\Program Files (x86)\Tesseract-OCR'

# Configurando o caminho para o executável do Tesseract-OCR e o idioma
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
tesseract_lang = 'eng'

# Função para extrair texto de um arquivo PDF usando OCR
def extrair_texto_ocr(caminho_arquivo):
    paginas = convert_from_path(caminho_arquivo)
    texto = ""
    for pagina in paginas:
        texto += pytesseract.image_to_string(pagina, lang=tesseract_lang)
    return texto

# Função para extrair texto de um arquivo PDF
def extrair_texto_pdf(caminho_arquivo):
    texto = ""
    with open(caminho_arquivo, 'rb') as arquivo:
        leitor = PdfReader(arquivo)
        for pagina in range(len(leitor.pages)):
            texto += leitor.pages[pagina].extract_text()
    return texto

# Diretórios de entrada e saída
diretorio_entrada = 'data/raw data'
diretorio_saida = 'data/nlp'

# Percorrer subpastas e arquivos PDF
for pasta_raiz, subpastas, arquivos in os.walk(diretorio_entrada):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            caminho_completo = os.path.join(pasta_raiz, arquivo)
            texto_extraido = extrair_texto_pdf(caminho_completo)
            if texto_extraido.count(' ') < 40:
                # Use OCR para identificar o texto em português
                texto_extraido = extrair_texto_ocr(caminho_completo)
            
            # Criar diretório de saída se não existir
            pasta_saida = pasta_raiz.replace(diretorio_entrada, diretorio_saida)
            os.makedirs(pasta_saida, exist_ok=True)
            # Escrever texto extraído em um arquivo de texto na pasta de saída
            nome_arquivo_saida = os.path.splitext(arquivo)[0] + '.txt'
            caminho_saida = os.path.join(pasta_saida, nome_arquivo_saida)
            with open(caminho_saida, 'w', encoding='utf-8') as arquivo_saida:
                arquivo_saida.write(texto_extraido)
